In [1]:
import ccxt
import pandas as pd
import numpy as np
import datetime

In [2]:
# Hyperparameters
START_DATE = '2024-01-01'
LOOKBACK_WINDOW = 7
NUMBER_OF_DAYS = 365
PAIR_NAME = 'BTC/USDT'

In [3]:
exchange = ccxt.binance()

def fetchOHLCV(symbol, timeframe, since):
	df = exchange.fetch_ohlcv(symbol=f'{symbol}', timeframe=timeframe, since=int(datetime.datetime.timestamp(since) * 1000), limit=NUMBER_OF_DAYS)
	df = pd.DataFrame(df, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
	df['Datetime'] = pd.to_datetime(df['timestamp'] * 1000 * 1000, utc=True).dt.tz_convert('Asia/Hong_Kong')
	df = df.set_index('Datetime')
	return df

start = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

df = fetchOHLCV(PAIR_NAME, '1d', start)
df

,timestamp,Open,High,Low,Close,Volume
Datetime,,,,,,
2024-01-01 08:00:00+08:00,1704067200000,42283.58,44184.10,42180.77,44179.55,27174.29903
2024-01-02 08:00:00+08:00,1704153600000,44179.55,45879.63,44148.34,44946.91,65146.40661
2024-01-03 08:00:00+08:00,1704240000000,44946.91,45500.00,40750.00,42845.23,81194.55173
2024-01-04 08:00:00+08:00,1704326400000,42845.23,44729.58,42613.77,44151.10,48038.06334
2024-01-05 08:00:00+08:00,1704412800000,44151.10,44357.46,42450.00,44145.11,48075.25327
...,...,...,...,...,...,...
2024-12-26 08:00:00+08:00,1735171200000,99429.61,99963.70,95199.14,95791.60,21192.36727
2024-12-27 08:00:00+08:00,1735257600000,95791.60,97544.58,93500.01,94299.03,26501.26429
2024-12-28 08:00:00+08:00,1735344000000,94299.03,95733.99,94135.66,95300.00,8385.89290


In [4]:
o = df['Open'].to_numpy()
h = df['High'].to_numpy()
l = df['Low'].to_numpy()
c = df['Close'].to_numpy()

In [5]:
from RelativeStrengthIndex import RelativeStrengthIndex

bt = RelativeStrengthIndex(LOOKBACK_WINDOW, -1)
for i in range(len(h)):
    bt.update(i, df.index, o, h, l, c)

pd.DataFrame(bt.trading_records)

""


In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from LocalRecord import calculate_equity_curve

records_df = pd.DataFrame(bt.trading_records)
equity_curve = calculate_equity_curve(records_df, df['Close'].pct_change())

# Create figure
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.35, row_heights=[0.7, 0.3])

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["Open"],
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    name="Candlestick"
), row=1, col=1)

# Add buy signals (green arrows)
fig.add_trace(go.Scatter(
    x=records_df["entry_timestamp"],
    y=records_df["entry_price"] * 0.9,
    mode="markers",
    marker=dict(symbol="triangle-up", color="green", size=10),
    name="Buy Signal"
), row=1, col=1)

# Add sell signals (red arrows)
fig.add_trace(go.Scatter(
    x=records_df["exit_timestamp"],
    y=records_df["exit_price"] * 1.1,
    mode="markers",
    marker=dict(symbol="triangle-down", color="red", size=10),
    name="Sell Signal"
), row=1, col=1)

# Add cumulative returns line chart
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Close'].pct_change().cumsum(),
    mode='lines',
    name='Equity Curve (%)',
    line=dict(color='blue')
), row=2, col=1)

# Add cumulative returns line chart
fig.add_trace(go.Scatter(
    x=df.index,
    y=equity_curve,
    mode='lines',
    name='Equity Curve (%)',
    line=dict(color='orange')
), row=2, col=1)

# Update layout
fig.update_layout(
    title="Candlestick Chart with Buy and Sell Signals",
    xaxis_title="Datetime",
    yaxis_title="Price",
    xaxis_rangeslider_visible=True,
    xaxis2_title="Datetime",
    yaxis2_title="Equity Curve (%)",
    xaxis2_rangeslider_visible=True,
    height=800
)

# Show figure
fig.show()

KeyError: 'entry_timestamp'

In [7]:
from tabulate import tabulate

total_trade = records_df.shape[0]
winners = (records_df['percentage_change'] > 0).sum()
losers = (records_df['percentage_change'] <= 0).sum()
win_ratio = round(winners/total_trade * 100, 2) if total_trade else 0
pnl = (records_df['percentage_change'] + 100) / 100
cumulative_return = round(pnl.prod() * 100 - 100, 2)
total_profit = records_df[records_df['percentage_change'] > 0]['percentage_change'].sum()
total_loss = records_df[records_df['percentage_change'] <= 0]['percentage_change'].sum()
avg_profit_per_trade = round(total_profit/winners, 2) if winners else 0
avg_loss_per_trade = round(total_loss/losers, 2) if losers else 0
avg_pnl_per_trade = round(cumulative_return/total_trade, 2) if total_trade else 0
risk_reward = f"1:{round(abs(avg_profit_per_trade/avg_loss_per_trade), 2)}" if avg_loss_per_trade else "N/A"

data = [
    ('Total Trade', total_trade),
    ('Cumulative Return', cumulative_return),
    ('Winners', winners),
    ('Losers', losers),
    ('% Win Ratio', win_ratio),
    ('% Average Profit per Trade', avg_profit_per_trade),
    ('% Average Loss per Trade', avg_loss_per_trade),
    ('% Average PNL per Trade', avg_pnl_per_trade),
    ('Risk Reward', risk_reward)
]

print(tabulate(data, headers=['Parameters', 'Values'], tablefmt='psql'))

+----------------------------+----------+
| Parameters                 | Values   |
|----------------------------+----------|
| Total Trade                | 2        |
| Cumulative Return          | 28.0     |
| Winners                    | 2        |
| Losers                     | 0        |
| % Win Ratio                | 100.0    |
| % Average Profit per Trade | 13.27    |
| % Average Loss per Trade   | 0        |
| % Average PNL per Trade    | 14.0     |
| Risk Reward                | N/A      |
+----------------------------+----------+
